In [1]:
import numpy as np
import torch
from torch import nn, optim, Tensor
import torch.utils.data as Data
import os
import sys
sys.path.append('../')
import utils.csv as csv
from models.transformer import TransformerRegression
import utils.validation as val

d:\LenovoSoftstore\Anaconda\envs\yolov5\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
# file path
PATH='D:\\Deutschland\\FUB\\master_thesis\\data\\gee\\output'
DATA_DIR = os.path.join(PATH, 'monthly_mean')
LABEL_CSV = '7_classes.csv'

label_path = os.path.join(PATH, LABEL_CSV)

In [43]:
# general hyperparameters
BATCH_SIZE = 128
LR = 0.01
EPOCH = 100
SEED = 12345

In [44]:
# hyperparameters for Transformer model
src_size = 8000
tgt_size = 101
d_model = 8
nhead = 4
num_layers = 1
dim_feedforward = 128

In [45]:
def numpy_to_tensor(x_data:np.ndarray, y_data:np.ndarray):
    # x_data = x_data.T
    # y_data = y_data.T
    x_set = torch.from_numpy(x_data)
    y_set = torch.from_numpy(y_data)
    return x_set, y_set

In [46]:
def build_dataloader(x_set:Tensor, y_set:Tensor, batch_size:int, seed:int):
    dataset = Data.TensorDataset(x_set, y_set)
    # split dataset
    size = len(dataset)
    train_size, val_size = round(0.8 * size), round(0.2 * size)
    generator = torch.Generator().manual_seed(seed)
    train_dataset, val_dataset = Data.random_split(dataset, [train_size, val_size], generator)
    # # manually split dataset
    # x_train = x_set[:444]
    # y_train = y_set[:444]
    # x_val = x_set[444:]
    # y_val = y_set[444:]
    # train_dataset = Data.TensorDataset(x_train, y_train)
    # val_dataset = Data.TensorDataset(x_val, y_val)
    # data_loader
    train_loader = Data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True,num_workers=2)
    val_loader = Data.DataLoader(val_dataset,batch_size=32, shuffle=True,num_workers=2)
    return train_loader, val_loader

In [47]:
def train(model:nn.Module, epoch:int):
    total_step = len(train_loader)
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.t().to(device)
        labels = labels.t().to(device)
        # forward pass
        outputs = model(inputs, labels)
        loss = criterion(outputs, labels/100.)
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # record training loss
        if i % 40 == 0:
            print('Epoch[{}/{}],Step[{}/{}],Loss:{:.4f}'
            .format(epoch+1,EPOCH,i+40,total_step,loss.item()))

In [ ]:
def validate(model:nn.Module):
    model.eval()
    good_pred = 0
    total = 0
    with torch.no_grad():
        for (inputs, labels) in val_loader:
            inputs = inputs.t().to(device)
            labels = labels.t().to(device)
            outputs = model(inputs, labels)
            print(outputs)
            print(outputs.size())
            good_pred += val.valid_num(labels/100., outputs)
            total += labels.size(1)
            break
    print(f'Validation accuracy: {good_pred / total * 100:.2f}%')

In [49]:
if __name__ == "__main__":
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # dataset
    x_data, y_data = csv.to_numpy(DATA_DIR, label_path)
    x_set, y_set = numpy_to_tensor(x_data, y_data)
    train_loader, val_loader = build_dataloader(x_set, y_set, BATCH_SIZE, SEED)
    # model
    model = TransformerRegression(src_size, tgt_size, d_model, nhead, num_layers, dim_feedforward).to(device)
    # loss and optimizer
    criterion = nn.MSELoss().to(device)
    optimizer = optim.SGD(model.parameters(), LR, momentum=0.99)
    # train and validate model
    print("Start training")
    for epoch in range(EPOCH):
        train(model, epoch)
        # validate(model)
    # save model
    # torch.save(model, '../outputs/model.pkl')

Load training data
Transfered data to numpy array
Start training
Epoch[1/100],Step[40/40],Loss:0.0676
Epoch[2/100],Step[40/40],Loss:0.0696
Epoch[3/100],Step[40/40],Loss:0.0647
Epoch[4/100],Step[40/40],Loss:0.0732


KeyboardInterrupt: 

In [50]:
validate(model)

tensor([[0.1620, 0.1910, 0.0912, 0.1571, 0.0818, 0.0917, 0.1354, 0.1537, 0.1143,
         0.1627, 0.1172, 0.1154, 0.0971, 0.1614, 0.1449, 0.0662, 0.1859, 0.1306,
         0.1567, 0.0831, 0.1936, 0.1678, 0.0962, 0.1426, 0.1516, 0.1411, 0.1072,
         0.1015, 0.1728, 0.1491, 0.1160, 0.1592],
        [0.1541, 0.1294, 0.1352, 0.1186, 0.1084, 0.0896, 0.2372, 0.1207, 0.1745,
         0.1092, 0.1081, 0.1756, 0.1119, 0.0781, 0.1435, 0.1432, 0.1413, 0.2005,
         0.1470, 0.1009, 0.1622, 0.2068, 0.1554, 0.1526, 0.0982, 0.1862, 0.1843,
         0.1610, 0.1126, 0.1669, 0.1569, 0.1214],
        [0.1184, 0.1371, 0.1638, 0.0931, 0.1547, 0.1332, 0.1104, 0.1535, 0.0953,
         0.1742, 0.1394, 0.1308, 0.1323, 0.2239, 0.1237, 0.1403, 0.1185, 0.1466,
         0.1281, 0.1268, 0.1253, 0.1953, 0.1966, 0.1695, 0.1905, 0.1532, 0.2183,
         0.1029, 0.1449, 0.1191, 0.1740, 0.1583],
        [0.1529, 0.1154, 0.1190, 0.1501, 0.2539, 0.2044, 0.0967, 0.1358, 0.1818,
         0.1243, 0.1592, 0.1639, 0.1304,